In [8]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np
from tqdm import tqdm

In [24]:
column_names = []
dfold = pd.DataFrame(columns = column_names)

for year in tqdm(range(6,10)):
    for month in tqdm(range(1,13)):
        if month < 10 :
                
            
            df = pd.read_excel("trimdata/apt200"+str(year)+"0"+str(month)+ ".xlsx" )

            # 이전파일 건드리지 않으려면 반드시 copy로
            df1 = df.copy()

            # datetime으로 만들기

            df1["년월"] = df1["계약년월"].copy()
            #format=%Y%m 으로 포맷 바꿔주기.
            df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

            # df1["년월"]
            # pd.to_datetime?
            # df1.info()


            # ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하기 
                # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
            df1["금액"] =df1["거래금액(만원)"].copy()
            df1["금액"] = df1["금액"].str.replace(",","")


            # 문자열인 값 숫자로 변환하기 
            df1["금액"] =pd.to_numeric(df1["금액"])
            df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
            df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


            #해제사유등 필요없는것 제거하기.

            # axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
            # 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
            df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
                        ,axis=1).copy()
            # df2


                
            #크기로 나누기
            df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
            df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
            df2.loc[(df2['면적'] > 85), '크기'] = '대형'
            #확인 
            df2['크기'].value_counts()


            #지역으로 나누기.
            df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
            df2["서울"].sum()
            df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
            df2["대전"].sum()

            location =[]
            location = ["강원도",     
                        "경기도"  ,  
                        "경상남도" ,    
                        "경상북도"  ,   
                        "광주광역"     ,
                        "대구광역"  ,
                        "대전광역"   ,
                        "부산광역"   ,
                        "서울특별"    ,
                        "세종특별",
                        "울산광역"    ,
                        "인천광역"    ,
                        "전라남도" ,    
                        "전라북도"  ,   
                        "제주특별"    ,
                        "충청남도" ,   
                        "충청북도"  ] 

            for i in location :
                df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
                df2.loc[(df2[i] ==  1) , '지역'] = i


            #확인
            df2["지역"].value_counts()
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


            #강남 송파 서초 3구 따로 묶어보기 ..abs
            df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
            df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
            df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
            df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

            #Nan 0으로 만들기
            df2["강남3구"] = df2["강남3구"].fillna(0).copy()

            #확인하기 
            df2["강남3구"].value_counts()


            # 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

            #결과 확인
            df2.info()

            # 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


            ###  설명 시계열 데이터 변환작업 


            df3 = df2.groupby(["지역","크기"])["금액"].mean().unstack().round()
            df3.columns = ["대형평균","소형평균","중형평균"]
            df3 = df3.reindex(columns=["소형평균","중형평균","대형평균"])
          #  df3


            df4 = df2.groupby(["지역","크기"])["금액"].std().unstack().round()
            df4.columns = ["대형표준편차","소형표준편차","중형표준편차"]
            df4 = df4.reindex(columns=["소형표준편차","중형표준편차","대형표준편차"])
          #  df4

            column_names = []
            df5 = pd.DataFrame(columns = column_names)
            df5["지역평균"] = df2.groupby(["지역"])["금액"].mean().round().copy()

            df6 = pd.DataFrame(columns = column_names)
            df6["지역표준편차"] = df2.groupby(["지역"])["금액"].std().round().copy()

            df_last = pd.concat([df5,df3,df6,df4] , axis=1) #axis=1을 해야 옆에다 붙임즉 column에 붙임. 디폴트는 axis=0 이고 이렇게하면 row에 붙이는것. 
           # df_last

            # 인덱스 초기화하기
            df_r = df_last.reset_index()
           # df_r 

            #전국 붙이기 
            nat = []
            nat =["전국"]
            nat.append(df2["금액"].mean())
            nat.append(df2.groupby(["크기"])["금액"].mean()["소형"])
            nat.append(df2.groupby(["크기"])["금액"].mean()["중형"])
            nat.append(df2.groupby(["크기"])["금액"].mean()["대형"])
            nat.append(df2["금액"].std())
            nat.append(df2.groupby(["크기"])["금액"].std()["소형"])
            nat.append(df2.groupby(["크기"])["금액"].std()["중형"])
            nat.append(df2.groupby(["크기"])["금액"].std()["대형"])

            # nat

            #시간축만들기. 여기 루프넣어야함. 
            df_r.loc[17] = nat

            df_r["년월"] = int("200"+str(year)+"0"+str(month))
            df_r["년월"] = pd.to_datetime(df_r["년월"], format='%Y%m', errors='ignore')

            
            df_r2=pd.concat([dfold,df_r])
            dfold = df_r2
        else : 

            df = pd.read_excel("trimdata/apt200"+str(year)+str(month)+ ".xlsx" )

            # 이전파일 건드리지 않으려면 반드시 copy로
            df1 = df.copy()

            # datetime으로 만들기

            df1["년월"] = df1["계약년월"].copy()
            #format=%Y%m 으로 포맷 바꿔주기.
            df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

            # df1["년월"]
            # pd.to_datetime?
            # df1.info()


            # ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하기 
                # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
            df1["금액"] =df1["거래금액(만원)"].copy()
            df1["금액"] = df1["금액"].str.replace(",","")


            # 문자열인 값 숫자로 변환하기 
            df1["금액"] =pd.to_numeric(df1["금액"])
            df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
            df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


            #해제사유등 필요없는것 제거하기.

            # axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
            # 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
            df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
                        ,axis=1).copy()
            # df2


                
            #크기로 나누기
            df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
            df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
            df2.loc[(df2['면적'] > 85), '크기'] = '대형'
            #확인 
            df2['크기'].value_counts()


            #지역으로 나누기.
            df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
            df2["서울"].sum()
            df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
            df2["대전"].sum()

            location =[]
            location = ["강원도",     
                        "경기도"  ,  
                        "경상남도" ,    
                        "경상북도"  ,   
                        "광주광역"     ,
                        "대구광역"  ,
                        "대전광역"   ,
                        "부산광역"   ,
                        "서울특별"    ,
                        "세종특별",
                        "울산광역"    ,
                        "인천광역"    ,
                        "전라남도" ,    
                        "전라북도"  ,   
                        "제주특별"    ,
                        "충청남도" ,   
                        "충청북도"  ] 

            for i in location :
                df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
                df2.loc[(df2[i] ==  1) , '지역'] = i


            #확인
            df2["지역"].value_counts()
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


            #강남 송파 서초 3구 따로 묶어보기 ..abs
            df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
            df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
            df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
            df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

            #Nan 0으로 만들기
            df2["강남3구"] = df2["강남3구"].fillna(0).copy()

            #확인하기 
            df2["강남3구"].value_counts()


            # 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

            #결과 확인
            df2.info()

            # 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
            df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


            ###  설명 시계열 데이터 변환작업 


            df3 = df2.groupby(["지역","크기"])["금액"].mean().unstack().round()
            df3.columns = ["대형평균","소형평균","중형평균"]
            df3 = df3.reindex(columns=["소형평균","중형평균","대형평균"])
          #  df3


            df4 = df2.groupby(["지역","크기"])["금액"].std().unstack().round()
            df4.columns = ["대형표준편차","소형표준편차","중형표준편차"]
            df4 = df4.reindex(columns=["소형표준편차","중형표준편차","대형표준편차"])
          #  df4

            column_names = []
            df5 = pd.DataFrame(columns = column_names)
            df5["지역평균"] = df2.groupby(["지역"])["금액"].mean().round().copy()

            df6 = pd.DataFrame(columns = column_names)
            df6["지역표준편차"] = df2.groupby(["지역"])["금액"].std().round().copy()

            df_last = pd.concat([df5,df3,df6,df4] , axis=1) #axis=1을 해야 옆에다 붙임즉 column에 붙임. 디폴트는 axis=0 이고 이렇게하면 row에 붙이는것. 
           # df_last

            # 인덱스 초기화하기
            df_r = df_last.reset_index()
           # df_r 

            #전국 붙이기 
            nat = []
            nat =["전국"]
            nat.append(df2["금액"].mean())
            nat.append(df2.groupby(["크기"])["금액"].mean()["소형"])
            nat.append(df2.groupby(["크기"])["금액"].mean()["중형"])
            nat.append(df2.groupby(["크기"])["금액"].mean()["대형"])
            nat.append(df2["금액"].std())
            nat.append(df2.groupby(["크기"])["금액"].std()["소형"])
            nat.append(df2.groupby(["크기"])["금액"].std()["중형"])
            nat.append(df2.groupby(["크기"])["금액"].std()["대형"])

            # nat

            #시간축만들기. 여기 루프넣어야함. 
            df_r.loc[17] = nat

            df_r["년월"] = int("200"+str(year)+str(month))
            df_r["년월"] = pd.to_datetime(df_r["년월"], format='%Y%m', errors='ignore')

            
            df_r2=pd.concat([dfold,df_r])
            dfold = df_r2            

  0%|          | 0/4 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   건축년도        25921 non-null  int64         
 3   년월          25921 non-null  datetime64[ns]
 4   금액          25921 non-null  int64         
 5   면적          25921 non-null  float64       
 6   크기          25921 non-null  object        
 7   서울          25921 non-null  int32         
 8   대전          25921 non-null  int32         
 9   강원도         25921 non-null  int32         
 10  지역          25921 non-null  object        
 11  경기도         25921 non-null  int32         
 12  경상남도        25921 non-null  int32         
 13  경상북도        25921 non-null  int32         
 14  광주광역        25921 non-null  int32         
 15  대구광역        25921 non-null  int32         
 16  대전광역        25921 non-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44742 entries, 0 to 44741
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  44742 non-null  int64         
 1   시군구         44742 non-null  object        
 2   건축년도        44742 non-null  int64         
 3   년월          44742 non-null  datetime64[ns]
 4   금액          44742 non-null  int64         
 5   면적          44742 non-null  float64       
 6   크기          44742 non-null  object        
 7   서울          44742 non-null  int32         
 8   대전          44742 non-null  int32         
 9   강원도         44742 non-null  int32         
 10  지역          44742 non-null  object        
 11  경기도         44742 non-null  int32         
 12  경상남도        44742 non-null  int32         
 13  경상북도        44742 non-null  int32         
 14  광주광역        44742 non-null  int32         
 15  대구광역        44742 non-null  int32         
 16  대전광역        44742 non-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58448 entries, 0 to 58447
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  58448 non-null  int64         
 1   시군구         58448 non-null  object        
 2   건축년도        58448 non-null  int64         
 3   년월          58448 non-null  datetime64[ns]
 4   금액          58448 non-null  int64         
 5   면적          58448 non-null  float64       
 6   크기          58448 non-null  object        
 7   서울          58448 non-null  int32         
 8   대전          58448 non-null  int32         
 9   강원도         58448 non-null  int32         
 10  지역          58448 non-null  object        
 11  경기도         58448 non-null  int32         
 12  경상남도        58448 non-null  int32         
 13  경상북도        58448 non-null  int32         
 14  광주광역        58448 non-null  int32         
 15  대구광역        58448 non-null  int32         
 16  대전광역        58448 non-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46590 entries, 0 to 46589
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  46590 non-null  int64         
 1   시군구         46590 non-null  object        
 2   건축년도        46590 non-null  int64         
 3   년월          46590 non-null  datetime64[ns]
 4   금액          46590 non-null  int64         
 5   면적          46590 non-null  float64       
 6   크기          46590 non-null  object        
 7   서울          46590 non-null  int32         
 8   대전          46590 non-null  int32         
 9   강원도         46590 non-null  int32         
 10  지역          46590 non-null  object        
 11  경기도         46590 non-null  int32         
 12  경상남도        46590 non-null  int32         
 13  경상북도        46590 non-null  int32         
 14  광주광역        46590 non-null  int32         
 15  대구광역        46590 non-null  int32         
 16  대전광역        46590 non-

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62342 entries, 0 to 62341
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  62342 non-null  int64         
 1   시군구         62342 non-null  object        
 2   건축년도        62342 non-null  int64         
 3   년월          62342 non-null  datetime64[ns]
 4   금액          62342 non-null  int64         
 5   면적          62342 non-null  float64       
 6   크기          62342 non-null  object        
 7   서울          62342 non-null  int32         
 8   대전          62342 non-null  int32         
 9   강원도         62342 non-null  int32         
 10  지역          62342 non-null  object        
 11  경기도         62342 non-null  int32         
 12  경상남도        62342 non-null  int32         
 13  경상북도        62342 non-null  int32         
 14  광주광역        62342 non-null  int32         
 15  대구광역        62342 non-null  int32         
 16  대전광역        62342 non-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49576 entries, 0 to 49575
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  49576 non-null  int64         
 1   시군구         49576 non-null  object        
 2   건축년도        49576 non-null  int64         
 3   년월          49576 non-null  datetime64[ns]
 4   금액          49576 non-null  int64         
 5   면적          49576 non-null  float64       
 6   크기          49576 non-null  object        
 7   서울          49576 non-null  int32         
 8   대전          49576 non-null  int32         
 9   강원도         49576 non-null  int32         
 10  지역          49576 non-null  object        
 11  경기도         49576 non-null  int32         
 12  경상남도        49576 non-null  int32         
 13  경상북도        49576 non-null  int32         
 14  광주광역        49576 non-null  int32         
 15  대구광역        49576 non-null  int32         
 16  대전광역        49576 non-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41800 entries, 0 to 41799
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  41800 non-null  int64         
 1   시군구         41800 non-null  object        
 2   건축년도        41800 non-null  int64         
 3   년월          41800 non-null  datetime64[ns]
 4   금액          41800 non-null  int64         
 5   면적          41800 non-null  float64       
 6   크기          41800 non-null  object        
 7   서울          41800 non-null  int32         
 8   대전          41800 non-null  int32         
 9   강원도         41800 non-null  int32         
 10  지역          41800 non-null  object        
 11  경기도         41800 non-null  int32         
 12  경상남도        41800 non-null  int32         
 13  경상북도        41800 non-null  int32         
 14  광주광역        41800 non-null  int32         
 15  대구광역        41800 non-null  int32         
 16  대전광역        41800 non-

100%|██████████| 4/4 [06:00<00:00, 90.24s/it]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45117 entries, 0 to 45116
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  45117 non-null  int64         
 1   시군구         45117 non-null  object        
 2   건축년도        45117 non-null  int64         
 3   년월          45117 non-null  datetime64[ns]
 4   금액          45117 non-null  int64         
 5   면적          45117 non-null  float64       
 6   크기          45117 non-null  object        
 7   서울          45117 non-null  int32         
 8   대전          45117 non-null  int32         
 9   강원도         45117 non-null  int32         
 10  지역          45117 non-null  object        
 11  경기도         45117 non-null  int32         
 12  경상남도        45117 non-null  int32         
 13  경상북도        45117 non-null  int32         
 14  광주광역        45117 non-null  int32         
 15  대구광역        45117 non-null  int32         
 16  대전광역        45117 non-

In [33]:
df_r2.tail(20)



,지역,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차,년월
16,충청북도,11422.000000,10018.000000,13185.000000,20892.000000,5404.000000,4405.000000,4549.000000,6601.000000,2009-11-01
17,전국,17231.684498,10652.883955,18361.622343,35650.704615,16257.420382,8333.461429,11942.606868,27585.681728,2009-11-01
0,강원도,10758.000000,5437.000000,12065.000000,24516.000000,7873.000000,2514.000000,4331.000000,6862.000000,2009-12-01
1,경기도,22566.000000,15209.000000,24702.000000,46634.000000,14541.000000,6777.000000,10820.000000,19813.000000,2009-12-01
2,경상남도,14446.000000,8106.000000,14492.000000,27487.000000,8560.000000,3381.000000,5234.000000,8969.000000,2009-12-01
3,경상북도,9403.000000,6219.000000,11545.000000,24244.000000,6427.000000,2162.000000,4426.000000,9442.000000,2009-12-01
4,광주광역,13381.000000,6936.000000,13647.000000,24371.000000,8178.000000,2055.000000,4499.000000,8494.000000,2009-12-01
5,대구광역,16371.000000,8630.000000,16262.000000,31721.000000,11220.000000,2593.000000,5050.000000,15031.000000,2009-12-01
6,대전광역,16123.000000,8775.000000,16706.000000,28408.000000,9291.000000,3269.000000,5186.000000,11350.000000,2009-12-01
7,부산광역,16995.000000,9668.000000,16368.000000,31132.000000,10623.000000,3932.000000,5841.000000,12512.000000,2009-12-01


In [35]:
  #수정된 파일 새경로에 저장하기
df_r2.to_excel(("timeseries/timeseries1.xlsx"))